In [8]:
#  ! pip install learn-to-pick

In [9]:
# To capture the new agent during dev
import sys

sys.path.append("../")

from autogen import UserProxyAgent
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
import learn_to_pick
import autogen

autogen.ChatCompletion.start_logging()

config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST", file_location="..")
llm_config = {
    "request_timeout": 60,
    "config_list": config_list,
    "use_cache": True,  # Use False to explore LLM non-determinism.
}

In [10]:
from autogen.agentchat.contrib.learn_to_pick_agent import LearnToPickAgent

ltp_config = {
    "selection_scorer": None,
}

ltp_agent = LearnToPickAgent(
    llm_config=llm_config,
    learn_to_pick_config=ltp_config,)

2023-11-02 21:11:47,288 - learn_to_pick.base - WARNING - No selection scorer provided, which means that no reinforcement learning will be done in the RL chain unless update_with_delayed_score is called.


In [11]:
user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=0,
)

In [12]:
# experimenting with different ways to shape the input
# all three below currently accepted

msg = {'content': {"to_select_from": {"name": "meal", "options": ["Beef", "Chicken", "Veggie"]}, "based_on": {"name": "user", "options": ["Meat eater", "loves beef"]}}}
msg = {'content': {"to_select_from": ["Beef", "Chicken", "Veggie"], "based_on": ["Meat eater", "loves beef"]}}
msg = {'content': {"meal": LearnToPickAgent.ToSelectFrom(["Beef", "Chicken", "Veggie"]), "user": LearnToPickAgent.BasedOn(["Meat eater", "loves beef"])}}

user.initiate_chat(ltp_agent, message=msg)

user (to learntopickagent):

{'meal': ['Beef', 'Chicken', 'Veggie'], 'user': ['Meat eater', 'loves beef']}

--------------------------------------------------------------------------------
learntopickagent (to user):

{'picked': {'meal': 'Chicken'}, 'picked_metadata': <learn_to_pick.pick_best.PickBestEvent object at 0x7f2f7c4f1280>}

--------------------------------------------------------------------------------
{'meal': ['Beef', 'Chicken', 'Veggie'], 'user': ['Meat eater', 'loves beef']}

--------------------------------------------------------------------------------
learntopickagent (to user):

{'picked': {'meal': 'Chicken'}, 'picked_metadata': <learn_to_pick.pick_best.PickBestEvent object at 0x7f2f7c4f1280>}

--------------------------------------------------------------------------------


In [13]:
ltp_response = list(user.chat_messages.values())[0][1]["content"]

In [14]:
# eval ltp_response and define score
dummy_score = 1.0
ltp_agent.learn_from_user_feedback(dummy_score, ltp_response)